# Train, Evaluate, and Publish Terminal Command Model

This notebook:
1. Trains Qwen3-0.6B with QLoRA on terminal command dataset
2. Evaluates the model while still in memory
3. Saves LoRA adapters locally and publishes to HuggingFace
4. Saves merged model locally and publishes to HuggingFace

**HuggingFace Repos:**
- Adapters: `{username}/qwen3-0.6b-terminal-instruct-lora`
- Merged: `{username}/qwen3-0.6b-terminal-instruct`

## Cell 1: Environment Setup and CUDA Verification

In [ ]:
import os
import torch
import warnings
warnings.filterwarnings('ignore')

os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["CUDA_LAUNCH_BLOCKING"] = "0"

print("=" * 50)
print("CUDA VERIFICATION")
print("=" * 50)
print("PyTorch Version:", torch.__version__)

if torch.cuda.is_available():
    print(f"✅ CUDA is available!")
    print(f"   GPU: {torch.cuda.get_device_name(0)}")
    print(f"   VRAM: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")
    torch.backends.cudnn.benchmark = True
    torch.backends.cudnn.enabled = True
    device = torch.device("cuda:0")
    torch.cuda.set_device(device)
    torch.cuda.empty_cache()
else:
    print("❌ CUDA is NOT available!")
    device = torch.device("cpu")

print("=" * 50)

## Cell 2: Import Libraries

In [ ]:
import json
import gc
from datetime import datetime
from pathlib import Path

from datasets import Dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    Trainer,
    DataCollatorForSeq2Seq
)
from peft import (
    LoraConfig,
    get_peft_model,
    prepare_model_for_kbit_training,
    PeftModel
)
from huggingface_hub import HfApi, login
from tqdm import tqdm

print("✅ All libraries imported successfully")

## Cell 3: Configuration

In [ ]:
# ============================================
# HUGGINGFACE CONFIGURATION
# ============================================
HF_USERNAME = "Eng-Elias"  # <-- Change this to your HuggingFace username
HF_TOKEN = None  # Will prompt if None, or set your token here

# Repository names
HF_REPO_ADAPTERS = f"{HF_USERNAME}/qwen3-0.6b-terminal-instruct-lora"
HF_REPO_MERGED = f"{HF_USERNAME}/qwen3-0.6b-terminal-instruct"

# ============================================
# MODEL & TRAINING CONFIGURATION
# ============================================
CONFIG = {
    # Model
    "base_model": "Qwen/Qwen3-0.6B",
    
    # Paths
    "train_data": "../dataset/generated/processed/train.json",
    "dev_data": "../dataset/generated/processed/dev.json",
    "test_data": "../dataset/generated/processed/test.json",
    "output_dir": "../outputs/terminal_command_model",
    "adapter_save_path": "../outputs/lora_adapters",
    "merged_model_path": "../outputs/merged_model",
    
    # Quantization (for 6GB VRAM)
    "load_in_4bit": True,
    "bnb_4bit_quant_type": "nf4",
    "bnb_4bit_use_double_quant": True,
    "bnb_4bit_compute_dtype": "float16",
    
    # LoRA Configuration
    "lora_r": 16,
    "lora_alpha": 32,
    "lora_dropout": 0.1,
    "target_modules": ["q_proj", "k_proj", "v_proj", "o_proj"],
    
    # Training Configuration - RTX 2060 Optimized
    "num_epochs": 3,
    "per_device_train_batch_size": 2,
    "per_device_eval_batch_size": 2,
    "gradient_accumulation_steps": 8,
    "learning_rate": 2e-4,
    "lr_scheduler_type": "cosine",
    "warmup_ratio": 0.1,
    "max_seq_length": 256,
    "logging_steps": 25,
    "eval_steps": 100,
    "save_steps": 200,
    "save_total_limit": 2,
    
    # Memory Optimizations
    "gradient_checkpointing": True,
    "fp16": True,
    "bf16": False,
    "optim": "paged_adamw_8bit",
    
    # Evaluation
    "max_new_tokens": 150,
}

print("=" * 50)
print("CONFIGURATION")
print("=" * 50)
print(f"HuggingFace Username: {HF_USERNAME}")
print(f"Adapters Repo: {HF_REPO_ADAPTERS}")
print(f"Merged Model Repo: {HF_REPO_MERGED}")
print("=" * 50)

## Cell 4: HuggingFace Login

In [ ]:
# Login to HuggingFace
if HF_TOKEN is None:
    print("Please enter your HuggingFace token (get it from https://huggingface.co/settings/tokens):")
    HF_TOKEN = input("Token: ")

login(token=HF_TOKEN)
print("✅ Logged in to HuggingFace")

## Cell 5: Load Dataset

In [ ]:
def load_json_dataset(filepath):
    """Load dataset from JSON file."""
    with open(filepath, 'r', encoding='utf-8') as f:
        data = json.load(f)
    return Dataset.from_list(data)

print("Loading datasets...")
train_dataset = load_json_dataset(CONFIG["train_data"])
eval_dataset = load_json_dataset(CONFIG["dev_data"])
test_dataset = load_json_dataset(CONFIG["test_data"])

print(f"✅ Train samples: {len(train_dataset)}")
print(f"✅ Eval samples: {len(eval_dataset)}")
print(f"✅ Test samples: {len(test_dataset)}")

print("\n📝 Sample data:")
print(json.dumps(train_dataset[0], indent=2))

## Cell 6: Load Model and Tokenizer

In [ ]:
print("Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(
    CONFIG["base_model"],
    trust_remote_code=True
)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

print(f"✅ Tokenizer loaded")

# Quantization config
print("\nConfiguring 4-bit quantization...")
bnb_config = BitsAndBytesConfig(
    load_in_4bit=CONFIG["load_in_4bit"],
    bnb_4bit_quant_type=CONFIG["bnb_4bit_quant_type"],
    bnb_4bit_use_double_quant=CONFIG["bnb_4bit_use_double_quant"],
    bnb_4bit_compute_dtype=torch.float16
)

print("Loading model with quantization...")
model = AutoModelForCausalLM.from_pretrained(
    CONFIG["base_model"],
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
    torch_dtype=torch.float16
)

print(f"✅ Model loaded: {CONFIG['base_model']}")
if torch.cuda.is_available():
    print(f"   VRAM used: {torch.cuda.memory_allocated(0) / 1024**3:.2f} GB")

## Cell 7: Setup LoRA

In [ ]:
print("Preparing model for LoRA training...")
model = prepare_model_for_kbit_training(
    model,
    use_gradient_checkpointing=CONFIG["gradient_checkpointing"]
)

lora_config = LoraConfig(
    r=CONFIG["lora_r"],
    lora_alpha=CONFIG["lora_alpha"],
    target_modules=CONFIG["target_modules"],
    lora_dropout=CONFIG["lora_dropout"],
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)

print("\n" + "=" * 50)
print("TRAINABLE PARAMETERS")
print("=" * 50)
model.print_trainable_parameters()

if torch.cuda.is_available():
    print(f"\nVRAM used after LoRA setup: {torch.cuda.memory_allocated(0) / 1024**3:.2f} GB")

## Cell 8: Data Preprocessing

In [ ]:
def format_instruction(sample):
    """Format sample into instruction prompt with EOS token."""
    instruction = sample["instruction"]
    input_text = sample.get("input", "")
    output = sample["output"]
    
    if input_text:
        prompt = f"### Instruction:\n{instruction}\n\n### Input:\n{input_text}\n\n### Response:\n{output}{tokenizer.eos_token}"
    else:
        prompt = f"### Instruction:\n{instruction}\n\n### Response:\n{output}{tokenizer.eos_token}"
    
    return prompt

def preprocess_function(examples):
    """Tokenize and prepare data for training."""
    prompts = [format_instruction({"instruction": inst, "input": inp, "output": out}) 
               for inst, inp, out in zip(examples["instruction"], 
                                          examples["input"], 
                                          examples["output"])]
    
    tokenized = tokenizer(
        prompts,
        truncation=True,
        max_length=CONFIG["max_seq_length"],
        padding="max_length",
        return_tensors=None
    )
    
    tokenized["labels"] = tokenized["input_ids"].copy()
    
    return tokenized

print("📝 Sample formatted prompt:")
print("-" * 50)
print(format_instruction(train_dataset[0]))
print("-" * 50)

print("\nTokenizing datasets...")
tokenized_train = train_dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=train_dataset.column_names,
    desc="Tokenizing train"
)

tokenized_eval = eval_dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=eval_dataset.column_names,
    desc="Tokenizing eval"
)

print(f"✅ Tokenized train: {len(tokenized_train)} samples")
print(f"✅ Tokenized eval: {len(tokenized_eval)} samples")

## Cell 9: Training Arguments and Trainer

In [ ]:
Path(CONFIG["output_dir"]).mkdir(parents=True, exist_ok=True)
Path(CONFIG["adapter_save_path"]).mkdir(parents=True, exist_ok=True)
Path(CONFIG["merged_model_path"]).mkdir(parents=True, exist_ok=True)

training_args = TrainingArguments(
    output_dir=CONFIG["output_dir"],
    num_train_epochs=CONFIG["num_epochs"],
    per_device_train_batch_size=CONFIG["per_device_train_batch_size"],
    per_device_eval_batch_size=CONFIG["per_device_eval_batch_size"],
    gradient_accumulation_steps=CONFIG["gradient_accumulation_steps"],
    learning_rate=CONFIG["learning_rate"],
    lr_scheduler_type=CONFIG["lr_scheduler_type"],
    warmup_ratio=CONFIG["warmup_ratio"],
    gradient_checkpointing=CONFIG["gradient_checkpointing"],
    fp16=CONFIG["fp16"],
    bf16=CONFIG["bf16"],
    optim=CONFIG["optim"],
    eval_strategy="steps",
    eval_steps=CONFIG["eval_steps"],
    save_strategy="steps",
    save_steps=CONFIG["save_steps"],
    save_total_limit=CONFIG["save_total_limit"],
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    logging_dir=f"{CONFIG['output_dir']}/logs",
    logging_steps=CONFIG["logging_steps"],
    report_to="none",
    remove_unused_columns=False,
    dataloader_pin_memory=True,
    dataloader_num_workers=0,
)

data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    padding=True,
    return_tensors="pt"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_eval,
    data_collator=data_collator,
)

print("✅ Trainer initialized")
print(f"   Effective batch size: {CONFIG['per_device_train_batch_size'] * CONFIG['gradient_accumulation_steps']}")

## Cell 10: Train Model

In [ ]:
print("=" * 50)
print("🚀 STARTING TRAINING")
print("=" * 50)
print(f"Start time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

if torch.cuda.is_available():
    torch.cuda.empty_cache()
    gc.collect()

try:
    train_result = trainer.train()
    
    print("\n" + "=" * 50)
    print("✅ TRAINING COMPLETED!")
    print("=" * 50)
    print(f"End time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"Training time: {train_result.metrics['train_runtime'] / 60:.2f} minutes")
    print(f"Training loss: {train_result.metrics['train_loss']:.4f}")
    
except Exception as e:
    print(f"❌ Training failed: {e}")
    raise

## Cell 11: Evaluate Model In-Memory

In [ ]:
def generate_command(instruction, input_text="", max_new_tokens=150):
    """Generate command from instruction."""
    if input_text:
        prompt = f"### Instruction:\n{instruction}\n\n### Input:\n{input_text}\n\n### Response:\n"
    else:
        prompt = f"### Instruction:\n{instruction}\n\n### Response:\n"
    
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=200).to(model.device)
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=False,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id
        )
    
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    if "### Response:" in response:
        response = response.split("### Response:")[-1].strip()
    response = response.split("### ")[0].strip()
    
    return response

def exact_match(pred, gold):
    return pred.strip() == gold.strip()

def fuzzy_match(pred, gold):
    pred_norm = ' '.join(pred.lower().split())
    gold_norm = ' '.join(gold.lower().split())
    return pred_norm == gold_norm or gold_norm in pred_norm or pred_norm in gold_norm

# Load test data
with open(CONFIG["test_data"], 'r', encoding='utf-8') as f:
    test_data = json.load(f)

single_os_tests = [t for t in test_data if t["input"] in ["[LINUX]", "[WINDOWS]", "[MAC]", ""]]
json_tests = [t for t in test_data if "JSON" in t["input"].upper()]

print("=" * 50)
print("🔍 EVALUATING MODEL IN-MEMORY")
print("=" * 50)

# Evaluate single OS commands
print("\n📊 Evaluating Single OS Commands...")
single_results = {"total": 0, "exact_match": 0, "fuzzy_match": 0}

sample_size = min(100, len(single_os_tests))  # Sample for speed
for sample in tqdm(single_os_tests[:sample_size], desc="Evaluating"):
    pred = generate_command(sample["instruction"], sample["input"])
    gold = sample["output"]
    
    single_results["total"] += 1
    if exact_match(pred, gold):
        single_results["exact_match"] += 1
    if fuzzy_match(pred, gold):
        single_results["fuzzy_match"] += 1

print("\n" + "=" * 50)
print("📊 IN-MEMORY EVALUATION RESULTS")
print("=" * 50)
print(f"Total evaluated: {single_results['total']}")
print(f"Exact match: {single_results['exact_match']} ({100*single_results['exact_match']/single_results['total']:.1f}%)")
print(f"Fuzzy match: {single_results['fuzzy_match']} ({100*single_results['fuzzy_match']/single_results['total']:.1f}%)")
print("=" * 50)

## Cell 12: Save LoRA Adapters Locally

In [ ]:
print("=" * 50)
print("💾 SAVING LORA ADAPTERS LOCALLY")
print("=" * 50)

adapter_path = CONFIG["adapter_save_path"]

# Save LoRA adapters
model.save_pretrained(adapter_path)
print(f"✅ LoRA adapters saved to: {adapter_path}")

# Save tokenizer
tokenizer.save_pretrained(adapter_path)
print(f"✅ Tokenizer saved to: {adapter_path}")

# Save training config
config_save_path = f"{adapter_path}/training_config.json"
with open(config_save_path, 'w') as f:
    json.dump(CONFIG, f, indent=2)
print(f"✅ Config saved to: {config_save_path}")

print("\n📁 Saved adapter files:")
for f in Path(adapter_path).iterdir():
    size_mb = f.stat().st_size / 1024 / 1024
    print(f"   {f.name}: {size_mb:.2f} MB")

## Cell 13: Publish LoRA Adapters to HuggingFace

In [ ]:
print("=" * 50)
print("🚀 PUBLISHING LORA ADAPTERS TO HUGGINGFACE")
print("=" * 50)
print(f"Repository: {HF_REPO_ADAPTERS}")

try:
    # Push adapters to HuggingFace
    model.push_to_hub(
        HF_REPO_ADAPTERS,
        commit_message="Upload LoRA adapters for terminal command generation",
        private=False  # Set to True if you want private repo
    )
    print("✅ LoRA adapters pushed!")
    
    # Push tokenizer
    tokenizer.push_to_hub(
        HF_REPO_ADAPTERS,
        commit_message="Upload tokenizer"
    )
    print("✅ Tokenizer pushed!")
    
    print(f"\n🔗 View at: https://huggingface.co/{HF_REPO_ADAPTERS}")
    
except Exception as e:
    print(f"❌ Failed to push adapters: {e}")

## Cell 14: Create and Save Merged Model

In [ ]:
print("=" * 50)
print("🔀 CREATING MERGED MODEL")
print("=" * 50)

# ============================================
# IMPORTANT: Quantization Mismatch Explanation
# ============================================
# During QLoRA training, the LoRA adapters learn to compensate for the 
# specific quantization errors of the 4-bit base model. When merging:
# 1. We load the base model in float16 (not quantized)
# 2. Apply and merge the LoRA adapters
# 3. Save the merged model in float16
#
# If you later load this merged model with 4-bit quantization, 
# the NEW quantization pattern differs from training, causing ~17% accuracy drop.
#
# SOLUTION: Load merged model in float16 (no quantization) for best accuracy,
# or accept the accuracy trade-off when using quantization for memory savings.
# ============================================

print("Step 1: Loading base model in float16 (not quantized)...")

# Clear GPU memory from training
del model
del trainer
torch.cuda.empty_cache()
gc.collect()

# Load base model in full precision for proper merging
base_model_for_merge = AutoModelForCausalLM.from_pretrained(
    CONFIG["base_model"],
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True
)
print("✅ Base model loaded in float16")

print("\nStep 2: Loading LoRA adapters...")
model_with_lora = PeftModel.from_pretrained(base_model_for_merge, adapter_path)
print("✅ LoRA adapters loaded")

print("\nStep 3: Merging adapters into base model...")
merged_model = model_with_lora.merge_and_unload()
print("✅ Adapters merged successfully!")

# ============================================
# Step 4: Verify merged model accuracy (float16, no re-quantization)
# ============================================
print("\nStep 4: Verifying merged model accuracy (float16)...")
print("-" * 50)

def generate_with_model(model_to_use, instruction, input_text=""):
    """Generate command using specified model."""
    if input_text:
        prompt = f"### Instruction:\n{instruction}\n\n### Input:\n{input_text}\n\n### Response:\n"
    else:
        prompt = f"### Instruction:\n{instruction}\n\n### Response:\n"
    
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=200).to(model_to_use.device)
    
    with torch.no_grad():
        outputs = model_to_use.generate(
            **inputs,
            max_new_tokens=CONFIG["max_new_tokens"],
            do_sample=False,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id
        )
    
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    if "### Response:" in response:
        response = response.split("### Response:")[-1].strip()
    response = response.split("### ")[0].strip()
    return response

# Quick verification on subset of test data
merged_results = {"total": 0, "exact_match": 0}
verification_size = min(50, len(single_os_tests))

for sample in tqdm(single_os_tests[:verification_size], desc="Verifying merged model"):
    pred = generate_with_model(merged_model, sample["instruction"], sample["input"])
    gold = sample["output"]
    merged_results["total"] += 1
    if pred.strip() == gold.strip():
        merged_results["exact_match"] += 1

merged_accuracy = 100 * merged_results["exact_match"] / merged_results["total"]
print(f"\n✅ Merged model (float16) accuracy: {merged_accuracy:.1f}%")

if merged_accuracy >= 90:
    print("   ✅ Merged model maintains high accuracy when loaded in float16!")
else:
    print("   ⚠️ Unexpected accuracy drop - check merging process")

print("\n" + "-" * 50)
print("📌 NOTE: When loading this merged model with 4-bit quantization,")
print("   expect ~15-20% accuracy drop due to quantization mismatch.")
print("   For best results, load in float16 without quantization.")
print("-" * 50)

# Step 5: Save merged model
print("\nStep 5: Saving merged model locally...")
merged_model_path = CONFIG["merged_model_path"]
merged_model.save_pretrained(merged_model_path)
tokenizer.save_pretrained(merged_model_path)
print(f"✅ Merged model saved to: {merged_model_path}")

print("\n📁 Merged model files:")
for f in Path(merged_model_path).iterdir():
    size_mb = f.stat().st_size / 1024 / 1024
    print(f"   {f.name}: {size_mb:.2f} MB")

## Cell 15: Publish Merged Model to HuggingFace

In [ ]:
print("=" * 50)
print("🚀 PUBLISHING MERGED MODEL TO HUGGINGFACE")
print("=" * 50)
print(f"Repository: {HF_REPO_MERGED}")

try:
    # Push merged model to HuggingFace
    merged_model.push_to_hub(
        HF_REPO_MERGED,
        commit_message="Upload merged model for terminal command generation",
        private=False
    )
    print("✅ Merged model pushed!")
    
    # Push tokenizer
    tokenizer.push_to_hub(
        HF_REPO_MERGED,
        commit_message="Upload tokenizer"
    )
    print("✅ Tokenizer pushed!")
    
    print(f"\n🔗 View at: https://huggingface.co/{HF_REPO_MERGED}")
    
    # Important usage note
    print("\n" + "=" * 50)
    print("⚠️ IMPORTANT: LOADING INSTRUCTIONS")
    print("=" * 50)
    print("For BEST accuracy (~97%), load WITHOUT quantization:")
    print("```python")
    print("model = AutoModelForCausalLM.from_pretrained(")
    print(f'    "{HF_REPO_MERGED}",')
    print("    torch_dtype=torch.float16,")
    print("    device_map=\"auto\"")
    print(")")
    print("```")
    print("\nWith 4-bit quantization, expect ~80-83% accuracy due to")
    print("quantization mismatch from QLoRA training.")
    print("=" * 50)
    
except Exception as e:
    print(f"❌ Failed to push merged model: {e}")

## Cell 16: Summary

In [ ]:
print("\n" + "=" * 60)
print("✅ ALL TASKS COMPLETED!")
print("=" * 60)

print("\n📦 LOCAL SAVES:")
print(f"   LoRA Adapters: {CONFIG['adapter_save_path']}")
print(f"   Merged Model: {CONFIG['merged_model_path']}")

print("\n🌐 HUGGINGFACE REPOS:")
print(f"   Adapters: https://huggingface.co/{HF_REPO_ADAPTERS}")
print(f"   Merged:   https://huggingface.co/{HF_REPO_MERGED}")

print("\n📊 EVALUATION RESULTS:")
print(f"   In-Memory (LoRA + 4-bit base): {100*single_results['exact_match']/single_results['total']:.1f}%")
print(f"   Merged Model (float16):        {merged_accuracy:.1f}%")

print("\n" + "=" * 60)
print("⚠️ QUANTIZATION MISMATCH - IMPORTANT NOTES")
print("=" * 60)
print("""
During QLoRA training, LoRA adapters learn to compensate for the specific
quantization errors of the 4-bit base model. This creates a mismatch when
loading models differently:

┌─────────────────────────────────────────────────────────────┐
│ Loading Method                          │ Expected Accuracy │
├─────────────────────────────────────────┼───────────────────┤
│ Base (4-bit) + LoRA adapters            │ ~97% (best)       │
│ Merged model (float16, no quantization) │ ~97% (best)       │
│ Merged model (re-quantized to 4-bit)    │ ~80-83% (degraded)│
└─────────────────────────────────────────┴───────────────────┘

RECOMMENDATIONS:
1. For production with limited VRAM: Use base model + LoRA adapters
2. For production with enough VRAM: Use merged model in float16
3. Avoid re-quantizing the merged model (causes accuracy loss)
""")

print("📌 NEXT STEPS:")
print("   1. Run 02_evaluate_all_sources.ipynb to compare all loading methods")
print("   2. Run 03_load_and_test_all.ipynb for interactive testing")
print("   3. Update model cards on HuggingFace with loading instructions")

print("\n" + "=" * 60)